In [2]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

# Define the full path to your model directory
output_dir = '/Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/model_save/cls-5tasks-bs150-bs8-lr5e6-epoch100'

# Load the correct tokenizer (RobertaTokenizer) and model (T5ForConditionalGeneration)
tokenizer = RobertaTokenizer.from_pretrained(output_dir)
model = T5ForConditionalGeneration.from_pretrained(output_dir)

print("Model and tokenizer loaded successfully from:", output_dir)


Model and tokenizer loaded successfully from: /Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/model_save/cls-5tasks-bs150-bs8-lr5e6-epoch100


# Update jupyter and ipywidgets in terminal
pip install jupyter ipywidgets --upgrade

In [5]:
import pandas as pd

# Define the path to your test data
test_data_path = "/Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/data/test/prompt-target.csv"  
test_df = pd.read_csv(test_data_path)

print("Test dataset loaded. Number of samples:", len(test_df))
print(test_df.head())  # Display first few rows to verify



Test dataset loaded. Number of samples: 2320
                                              prompt  target
0  Predict the Tg of the following SMILES: *CC(C)...  119.85
1  Predict the Tg of the following SMILES: C(C(C)...  119.85
2  Predict the Tg of the following SMILES: CC(*)(...  119.85
3  Predict the Tg of the following SMILES: *C(C*)...  119.85
4  Predict the Tg of the following SMILES: *C(C#N...  119.85


In [13]:
# Initialize a list to store predictions
predictions = []

# Define device and move model to device if applicable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i, row in test_df.iterrows():
    input_text = row['prompt']  
    generated = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate prediction with updated parameters
    sample_outputs = model.generate(
        generated,
        do_sample=False,         # Use deterministic sampling
        top_k=100,               # Limit sampling to top 100 tokens
        max_length=50,           # Set a reasonable max length for generation
        top_p=0.99,              # Enable nucleus sampling with cumulative probability
        num_return_sequences=1   # Generate a single sequence
    )
    
    # Decode and add prediction to list
    prediction = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    predictions.append(prediction)

    # Display progress every 10 sa



/opt/anaconda3/envs/polyNC1/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.99` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/anaconda3/envs/polyNC1/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [14]:
from nltk.translate.bleu_score import sentence_bleu

# Calculate BLEU scores for each prediction
bleu_scores = []
for i, row in test_df.iterrows():
    reference = [row['target'].split()]  
    candidate = row['predicted_output'].split()
    bleu_score = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu_score)

# Add BLEU scores to DataFrame
test_df['bleu_score'] = bleu_scores
average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", average_bleu)



Average BLEU score: 0.0


In [16]:
sample_outputs = model.generate(
    generated,
    max_length=50,       # Increase length if necessary
    num_return_sequences=1
)





In [17]:
input_text = test_df.loc[0, 'prompt']  # Replace 'prompt' with the actual column name
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

sample_outputs = model.generate(input_ids, max_length=50)
prediction = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
print("Single Prediction Output:", prediction)


Single Prediction Output: 


In [ ]:
sample_outputs = model.generate(
    generated,
    max_length=50,       # Increase length if necessary
    num_return_sequences=1
)



